In [71]:
from bs4 import BeautifulSoup
import re
import requests
import operator
import threading
from threading import Thread
import pickle

In [3]:
%ls -lt

total 32
-rw-r--r--   1 bala  staff  6785 Nov 19 10:22 File-Pattern-CVE.ipynb
-rw-r--r--   1 bala  staff  2379 Nov 18 12:45 scrape.py
-rw-r--r--   1 bala  staff    36 Nov 18 08:54 README.md
drwxr-xr-x  31 bala  staff   992 Nov 15 22:10 cve-data/


In [4]:
file_base = %pwd
file_base += "/cve-data/"
file_base

'/Users/bala/Desktop/227/code/CSE-227/cve-data/'

In [52]:
CVE_IDS = []
file = "download ("
for i in range(0,29):
    file_path = file_base + file + str(i) + ").tsv"
    with open(file_path) as f:
        contents = f.readlines()
        for line in contents[1:]:
            words = line.split('\t')
            CVE_IDS.append(words[1])

In [53]:
CVE_TO_BUGID = dict()

for cve in CVE_IDS:
    url = "http://www.cvedetails.com/cve/" + cve + "/"
    hdr = {'User-Agent': 'Mozilla/5.0'}
    html = requests.get(url)
    soup = BeautifulSoup(html.text, "html.parser")
    
    table = soup.find("table", {"id": "vulnrefstable"})
    a_tags = table.find_all('a')
    for a in a_tags:
        if "bugzilla" in a['href']:
            if cve in CVE_TO_BUGID:
                CVE_TO_BUGID[cve].append(a['href'].split("=")[1])
            else:
                CVE_TO_BUGID[cve] = [a['href'].split("=")[1]]

# some CVE entries don't have bugzilla links
print(len(CVE_TO_BUGID))

1233


In [54]:
FILE_FREQUENCY = dict()
for key_cve, value_bugs in CVE_TO_BUGID.items():
    print(key_cve, len(value_bugs))
    for bug in value_bugs:
        s_url = "https://www.openhub.net/p/firefox/commits?query="+ str(bug)
        s_hdr = {'User-Agent': 'Mozilla/5.0'}
        s_html = requests.get(s_url)
        s_soup = BeautifulSoup(s_html.text, "html.parser")

        tags = s_soup('a')
        for t in tags:
            l = t.get('href', None)
            if l != None:
                commit = re.findall("commits/[0-9]+", l)
                if not commit:
                    continue
                t_url = "https://www.openhub.net/p/firefox/commits/" + commit[0].split("/")[1]
                t_hdr = {'User-Agent': 'Mozilla/5.0'}
                t_html = requests.get(t_url)
                t_soup = BeautifulSoup(t_html.text, "html.parser")
                rows = t_soup.findAll('tr', {'class': 'odd nohover'})
                if rows != []:
                    for row in rows:
                        file = row.find_all('td')[0].contents[0].strip()
                        if file in FILE_FREQUENCY:
                            FILE_FREQUENCY[file] += 1
                        else:
                            FILE_FREQUENCY[file] = 1
                                

CVE-2016-5284 1
CVE-2016-5283 1
CVE-2016-5282 1
CVE-2016-5281 1
CVE-2016-5280 1
CVE-2016-5279 1
CVE-2016-5278 1
CVE-2016-5277 1
CVE-2016-5276 1
CVE-2016-5275 1
CVE-2016-5274 1
CVE-2016-5273 1
CVE-2016-5272 1
CVE-2016-5271 1
CVE-2016-5270 1
CVE-2016-5268 1
CVE-2016-5267 1
CVE-2016-5266 1
CVE-2016-5265 1
CVE-2016-5264 1
CVE-2016-5263 1
CVE-2016-5262 1
CVE-2016-5261 1
CVE-2016-5260 1
CVE-2016-5259 1
CVE-2016-5258 1
CVE-2016-5257 9
CVE-2016-5256 7
CVE-2016-5255 1
CVE-2016-5254 1
CVE-2016-5253 1
CVE-2016-5252 1
CVE-2016-5251 1
CVE-2016-5250 1
CVE-2016-2839 1
CVE-2016-2838 1
CVE-2016-2837 1
CVE-2016-2836 8
CVE-2016-2835 9
CVE-2016-2834 4
CVE-2016-2833 1
CVE-2016-2832 1
CVE-2016-2831 1
CVE-2016-2830 1
CVE-2016-2829 1
CVE-2016-2828 1
CVE-2016-2827 1
CVE-2016-2826 1
CVE-2016-2825 1
CVE-2016-2824 1
CVE-2016-2822 1
CVE-2016-2821 1
CVE-2016-2820 1
CVE-2016-2819 1
CVE-2016-2818 13
CVE-2016-2817 1
CVE-2016-2816 1
CVE-2016-2815 6
CVE-2016-2814 1
CVE-2016-2813 1
CVE-2016-2812 1
CVE-2016-2811 1
CVE-201

CVE-2013-0795 1
CVE-2013-0794 1
CVE-2013-0793 1
CVE-2013-0792 1
CVE-2013-0791 1
CVE-2013-0790 1
CVE-2013-0789 13
CVE-2013-0788 10
CVE-2013-0787 1
CVE-2013-0784 12
CVE-2013-0783 10
CVE-2013-0782 1
CVE-2013-0781 1
CVE-2013-0780 1
CVE-2013-0779 1
CVE-2013-0778 1
CVE-2013-0777 1
CVE-2013-0776 1
CVE-2013-0775 1
CVE-2013-0774 1
CVE-2013-0773 1
CVE-2013-0772 1
CVE-2013-0771 1
CVE-2013-0770 4
CVE-2013-0769 5
CVE-2013-0768 1
CVE-2013-0767 1
CVE-2013-0766 1
CVE-2013-0765 1
CVE-2013-0764 1
CVE-2013-0763 1
CVE-2013-0762 1
CVE-2013-0761 1
CVE-2013-0760 1
CVE-2013-0759 1
CVE-2013-0758 1
CVE-2013-0757 1
CVE-2013-0756 1
CVE-2013-0755 1
CVE-2013-0754 1
CVE-2013-0753 1
CVE-2013-0752 1
CVE-2013-0751 1
CVE-2013-0750 1
CVE-2013-0749 9
CVE-2013-0748 1
CVE-2013-0747 1
CVE-2013-0746 1
CVE-2013-0745 1
CVE-2013-0744 1
CVE-2012-5843 14
CVE-2012-5842 5
CVE-2012-5841 1
CVE-2012-5840 1
CVE-2012-5839 1
CVE-2012-5838 1
CVE-2012-5837 1
CVE-2012-5836 1
CVE-2012-5835 1
CVE-2012-5833 1
CVE-2012-5830 1
CVE-2012-5829 1
CVE

CVE-2009-0356 1
CVE-2009-0355 1
CVE-2009-0354 1
CVE-2009-0353 1
CVE-2009-0352 11
CVE-2009-0071 3
CVE-2008-7293 1
CVE-2008-5913 1
CVE-2008-5512 2
CVE-2008-5511 2
CVE-2008-5510 1
CVE-2008-5508 2
CVE-2008-5507 1
CVE-2008-5506 1
CVE-2008-5505 1
CVE-2008-5504 1
CVE-2008-5503 1
CVE-2008-5502 1
CVE-2008-5501 1
CVE-2008-5500 2
CVE-2008-5052 1
CVE-2008-5024 1
CVE-2008-5023 1
CVE-2008-5022 1
CVE-2008-5021 1
CVE-2008-5019 1
CVE-2008-5018 1
CVE-2008-5017 1
CVE-2008-5016 1
CVE-2008-5015 1
CVE-2008-5014 1
CVE-2008-5013 1
CVE-2008-5012 2
CVE-2008-4582 1
CVE-2008-4069 1
CVE-2008-4067 2
CVE-2008-4066 1
CVE-2008-4065 1
CVE-2008-4064 3
CVE-2008-4063 3
CVE-2008-4062 3
CVE-2008-4061 1
CVE-2008-4060 2
CVE-2008-4059 1
CVE-2008-4058 2
CVE-2008-3837 1
CVE-2008-3836 2
CVE-2008-3835 1
CVE-2008-3444 1
CVE-2008-3198 1
CVE-2008-2934 1
CVE-2008-2933 1
CVE-2008-2811 1
CVE-2008-2810 1
CVE-2008-2809 3
CVE-2008-2808 1
CVE-2008-2807 1
CVE-2008-2806 1
CVE-2008-2805 1
CVE-2008-2803 1
CVE-2008-2802 1
CVE-2008-2801 3
CVE-200

In [55]:
print(len(FILE_FREQUENCY))

1875


In [73]:
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [74]:
#multithreading experiment
#lock = threading.Lock()

#def main_scraper():
#    Thread(target = worker).start()
#    Thread(target = worker).start()
#    Thread(target = worker).start()

# lock.acquire()
# try:
#     // code
# finally:
#     lock.release()

#storing the dict
save_obj(FILE_FREQUENCY, "file_freq")

#retrival of dict
tt = load_obj("file_freq")
sorted_FF = sorted(tt.items(), key=operator.itemgetter(1), reverse=True)
sorted_FF[1:30]

In [76]:
sorted_FF = sorted(FILE_FREQUENCY.items(), key=operator.itemgetter(1), reverse=True)
sorted_FF[1:20]

[('layout/generic/crashtests/crashtests.list', 35),
 ('content/base/src/nsContentUtils.cpp', 28),
 ('content/canvas/src/WebGLContext.h', 21),
 ('layout/base/crashtests/crashtests.list', 16),
 ('dom/base/nsGlobalWindow.h', 16),
 ('js/src/vm/Stack.cpp', 14),
 ('browser/base/content/test/general/browse...', 13),
 ('docshell/base/nsDocShell.h', 13),
 ('content/canvas/src/CanvasRenderingContex...', 13),
 ('docshell/base/nsDocShell.cpp', 13),
 ('dom/base/nsJSEnvironment.cpp', 13),
 ('layout/forms/nsComboboxControlFrame.h', 12),
 ('dom/imptests/failures/editing/selecttest...', 10),
 ('js/src/jsscope.h', 10),
 ('js/src/jsscript.cpp', 9),
 ('js/src/jsgc.cpp', 9),
 ('js/src/jsobj.h', 9),
 ('js/xpconnect/src/xpcprivate.h', 9),
 ('js/src/jsobj.cpp', 9)]